In [1]:
# Get resources
import numpy as np
import sys
sys.path.append('../../../../Programs/MieStuff/Code - sspstuff/ssp_pmom_pcode/')
import pmomstuff4 as pm
import scipy.io
import copy
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [20]:
# Get filename -- make sure temperatures in paths are correct
ncfilename = "ssp_getpsd_BL298_S331_pmom.nc"
ncfilename_truncated = "ssp_getpsd_BL298_S331_pmom_trunc.nc"

In [4]:
# Get the netcdf data
Npmomarray, pmomarray, \
wnum_mesh, reff_mesh, w0_mesh, qext_mesh, asym_mesh, \
wnum_list, reff_list, maxdim_list, volume_list, parea_list = \
pm.pmomload2(ncfilename)

In [5]:
# Report out dimensions
print (Npmomarray.shape)
print (pmomarray.shape)
print (wnum_mesh.shape)
print (reff_mesh.shape)
print (w0_mesh.shape)
print (qext_mesh.shape)
print (asym_mesh.shape)
print (wnum_list.shape)
print (reff_list.shape)
print (maxdim_list.shape)
print (volume_list.shape)
print (parea_list.shape)

# Report & make local copies of dimension information
N_reff, N_wnum, N_maxpmom = pmomarray.shape; print (N_reff, N_wnum, N_maxpmom)

(40, 4017)
(40, 4017, 1000)
(40, 4017)
(40, 4017)
(40, 4017)
(40, 4017)
(40, 4017)
(4017, 1)
(40, 1)
(40, 1)
(40, 1)
(40, 1)
40 4017 1000


In [16]:
# Decide on a new # of pmoms to max out at
N_maxpmom_new = 25 # This is 1st one (always = 1) plus g1, g2, ... 

# Make new versions of arrays that depend on # of pmoms
pmomarray_new = pmomarray[:,:,0:N_maxpmom_new]
Npmomarray_new = copy.deepcopy(Npmomarray)

# Report new dimension information
print (pmomarray_new.shape)

(40, 4017, 25)


In [17]:
# This is some QC: Fixing any missing data and limiting Npmomarray to the maximum
N_truncated = 0
N_zerofix = 0
for i_reff in range(0,N_reff):
    for i_wnum in range(0,N_wnum):
        N_fromfortran = Npmomarray_new[i_reff,i_wnum]
        
        if (N_fromfortran == 0):
            Npmomarray_new[i_reff,i_wnum] = 1 # Should be 1 regardless
            pmomarray_new[i_reff,i_wnum,0] = 1.0 # Should be 1.0 regardless
            N_zerofix += 1
            
        if (N_fromfortran > N_maxpmom_new):
            Npmomarray_new[i_reff,i_wnum] = N_maxpmom_new
            N_truncated += 1
print ("# zero-fixed = ", N_zerofix)
print ("# truncated = ", N_truncated)

# zero-fixed =  581
# truncated =  131708


In [18]:
# This is some double-checking that we did the above right
N_truncated = 0
N_zerofix = 0
for i_reff in range(0,N_reff):
    for i_wnum in range(0,N_wnum):
        N_fromfortran = Npmomarray_new[i_reff,i_wnum]
        
        if (N_fromfortran == 0):
            Npmomarray_new[i_reff,i_wnum] = 1 # Should be 1 regardless
            pmomarray_new[i_reff,i_wnum,0] = 1.0 # Should be 1.0 regardless
            N_zerofix += 1
            
        if (N_fromfortran > N_maxpmom_new):
            Npmomarray_new[i_reff,i_wnum] = N_maxpmom_new
            N_truncated += 1
print ("# zero-fixed = ", N_zerofix)
print ("# truncated = ", N_truncated)

# zero-fixed =  0
# truncated =  0


In [21]:
# Save the moments as a netcdf file
pm.pmomsave2(\
                 ncfilename_truncated,\
                 Npmomarray_new,pmomarray_new,\
                 wnum_mesh, reff_mesh, w0_mesh, qext_mesh, asym_mesh,\
                 wnum_list, reff_list, maxdim_list, volume_list, parea_list)